## Aplicar as técnicas seguintes:

•	Random Forest (RF) 

•	ExtraTreesClassiﬁer (ETC)

•	XGBoost (XGB) com Early Stopping

•	Gaussian Naive Bayes


## Preparando os dados para treinar

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

from sklearn import metrics

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.naive_bayes import GaussianNB

from xgboost.sklearn import XGBClassifier

## importando os dataframes para treino

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
df3 = pd.read_csv('/content/drive/Shared drives/iniciacao_cientifica_2019_2020/dataframes_treino/turma_102_2016_1/Dataframe3_Turma_102_2016_06_07_12_00_00_2016_07_28_12_00_00.csv', sep=';')

In [5]:
df_target_user = pd.read_csv('/content/drive/Shared drives/iniciacao_cientifica_2019_2020/csv_datasets/usuarios_classificados.csv')

## Criando um merge entre o df_target_user e df1

In [6]:
df_target_user = df_target_user[['semester_id', 'class_id', 'user_id', 'target']]

In [7]:
df3_data_target = pd.merge(df3, df_target_user, on=['semester_id', 'class_id', 'user_id'], how='inner')

In [8]:
df3_data_target.drop(['semester_id', 'class_id', 'user_id'], axis=1, inplace=True)

In [9]:
df3_data_target

,number_logins,average_active_time,average_exam_sub_comp,average_exam_sub_inc,average_exam_sub_err,average_exam_test,average_exam_test_err,average_exam_code_line,average_exam_sub_average_interval,average_homework_sub_comp,average_homework_sub_inc,average_homework_sub_err,average_homework_test,average_homework_test_err,average_homework_code_line,average_homework_test_average,target
0,43,160.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,2.000,0.000,44.000,158.000,Desistente
1,24,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,194.500,0.000,Desistente
2,44,589.766,1.167,8.667,1.333,21.000,3.333,1270.000,128.566,0.756,8.854,1.707,2.171,0.878,659.220,113.977,Não Desistente
3,10,168.600,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,1.000,0.000,0.000,0.000,46.000,0.000,Desistente
4,8,530.667,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.250,0.333,0.417,0.000,0.000,53.250,0.000,Desistente
5,35,544.125,0.667,5.917,3.667,7.917,1.500,791.583,23.292,1.309,3.044,1.588,2.294,0.794,829.441,1110.621,Não Desistente
6,25,375.243,3.273,15.818,2.636,5.273,0.727,1582.091,10.850,0.460,0.651,0.333,2.143,1.127,214.317,13.386,Não Desistente
7,37,479.325,0.667,3.500,1.417,3.333,1.833,964.667,20.581,0.794,2.441,0.868,2.147,0.412,668.015,1466.764,Não Desistente
8,48,531.538,0.500,1.500,0.750,6.500,1.250,714.833,55.259,0.838,2.985,0.794,4.368,1.191,336.706,390.482,Não Desistente
9,73,481.788,2.083,0.417,0.583,3.583,0.333,319.583,113.596,0.897,2.015,0.338,3.838,0.515,522.029,269.405,Não Desistente


## Preparando para treinar os dados até a primeira avaliação

In [11]:
# pegando as colunas de features
features = df3_data_target.columns.difference(['target'])

# dados de treino
X = df3_data_target[features].values

# classe target
y = df3_data_target['target'].values

## Dividindo uma parte para teste e outra para treino

In [12]:
# dividindo uma parte dos dados para treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [13]:
# verificando a forma dos dados de treino
X_train.shape, y_train.shape

((37, 16), (37,))

In [14]:
# verificando a forma dos dados de teste
X_test.shape, y_test.shape

((16, 16), (16,))

## Utilizando um GridSearchCV e RandomForestClassifier para encontrar os melhores parâmetros

In [15]:
# criando uma instância de RFC
classifier_rf = RandomForestClassifier()

# parâmetros para o gridsearchcv
param_grid_ = {
    "criterion": ['entropy', 'gini'],
    "n_estimators": [25, 50, 75],
    "bootstrap": [False, True],
    "max_depth": [3, 5, 10]
} 

# criando uma instância do gridsearchcv
grid_search = GridSearchCV(
    estimator=classifier_rf, 
    param_grid=param_grid_, 
    scoring="accuracy", 
    cv=5
)

# treinando com o gridsearch
grid_search.fit(X_train, y_train)

# atrinbuindo o modelo
classifier_rf = grid_search.best_estimator_ 

print(grid_search.best_params_, grid_search.best_score_)

{'bootstrap': True, 'criterion': 'entropy', 'max_depth': 3, 'n_estimators': 75} 0.95


In [16]:
# classificando o conjunto de teste
y_predict = classifier_rf.predict(X_test)

# metricas de avaliação de desempenho
print(metrics.classification_report(y_test, y_predict))

# matrix de confusão
print(metrics.confusion_matrix(y_test, y_predict))

                precision    recall  f1-score   support

    Desistente       1.00      0.75      0.86         4
Não Desistente       0.92      1.00      0.96        12

      accuracy                           0.94        16
     macro avg       0.96      0.88      0.91        16
  weighted avg       0.94      0.94      0.93        16

[[ 3  1]
 [ 0 12]]


## Encontrando os melhores parâmetros com GridSearch o ExtraTreesClassifier

In [17]:
# criando uma instância de ETC
classifier_et = ExtraTreesClassifier()

# parametros para o gridsearchcv
param_grid_ = {
    "criterion": ['entropy', 'gini'],
    "n_estimators": [25, 50, 75],
    "bootstrap": [False, True],
    "max_depth": [3, 5, 10]
} 

# criando uma instância de gridsearchcv
grid_search = GridSearchCV(
    estimator=classifier_et, 
    param_grid = param_grid_, 
    scoring="accuracy", 
    cv=5
)

# treinando com os dados de treino
grid_search.fit(X_train, y_train)

# atribuindo a instância os melhores parâmetros
classifier_et = grid_search.best_estimator_ 

print(grid_search.best_params_, grid_search.best_score_)

{'bootstrap': False, 'criterion': 'entropy', 'max_depth': 3, 'n_estimators': 50} 0.95


In [18]:
# classificando o conjunto de teste
y_predict = classifier_et.predict(X_test)

# metricas de avaliação de desempenho
print(metrics.classification_report(y_test, y_predict))

# matrix de confusão
print(metrics.confusion_matrix(y_test, y_predict))

                precision    recall  f1-score   support

    Desistente       1.00      0.75      0.86         4
Não Desistente       0.92      1.00      0.96        12

      accuracy                           0.94        16
     macro avg       0.96      0.88      0.91        16
  weighted avg       0.94      0.94      0.93        16

[[ 3  1]
 [ 0 12]]


## XGB e GridSearchCV para encontrar os melhores parâmetros

### Parâmetros
**num_boost_rounds:** corresponde ao número de rodadas de aumento ou árvores a serem construídas.

**early_stopping_round:** verifica se nas N primeiras rodadas o algoritmo melhora, caso não melhore ele para e fica com a melhor rodada. É nessário usar o **eval_set**

**eval_set:** (list, opcional) - Uma lista de pares de tuplas (X, y) a serem usados ​​como conjuntos de validação, para os quais as métricas serão calculadas. As métricas de validação nos ajudarão a acompanhar o desempenho do modelo.

**max_depth:** é o número máximo de nós permitido da raiz até a folha mais distante de uma árvore.

**min_child_weight:** é o peso mínimo (ou número de amostras, se todas as amostras tiverem um peso de 1) necessário para criar um novo nó na árvore.

**subsample:** corresponde à fração de observações (as linhas) para subamostra em cada etapa. Por padrão, é definido como 1, o que significa que usamos todas as linhas.

**colsample_bytree:** corresponde à fração de recursos (as colunas) a serem usadas. Por padrão, é definido como 1, o que significa que usaremos todos os recursos.

**ETA:** controla a taxa de aprendizado. (com um eta mais baixo, precisamos de mais rodadas de reforço)

In [19]:
# instanciando o xgbclassifier com algumas características
classifier_xgb = XGBClassifier(
    objective= 'binary:logistic',
    nthread=4,
    seed=42
)

# setando os parâmetros usados no grid
parameters = {
    'max_depth': range (2, 10, 1),
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05]
}

# setando os parâmetros do GridSearchCV
grid_search = GridSearchCV(
    estimator=classifier_xgb,
    param_grid=parameters,
    scoring = 'accuracy',
    n_jobs = 10,
    cv = 10,
    # verbose=True
)

fit_params={
    'early_stopping_rounds':10,
    'eval_set': [(X_test, y_test)],
    'verbose':True
}

# treinando com o melhor estimator
grid_search.fit(
    X_train, 
    y_train,
    **fit_params
)

# atribuindo a instância os melhores parâmetros
classifier_xgb = grid_search.best_estimator_ 

print(grid_search.best_params_, grid_search.best_score_)

[0]	validation_0-error:0.0625
Will train until validation_0-error hasn't improved in 10 rounds.
[1]	validation_0-error:0.0625
[2]	validation_0-error:0.0625
[3]	validation_0-error:0.0625
[4]	validation_0-error:0.0625
[5]	validation_0-error:0.0625
[6]	validation_0-error:0.0625
[7]	validation_0-error:0.0625
[8]	validation_0-error:0.0625
[9]	validation_0-error:0.0625
[10]	validation_0-error:0.0625
Stopping. Best iteration:
[0]	validation_0-error:0.0625

{'learning_rate': 0.1, 'max_depth': 2, 'n_estimators': 60} 0.9


In [20]:
# classificando o conjunto de teste
y_predict = classifier_xgb.predict(X_test)

# metricas de avaliação de desempenho
print(metrics.classification_report(y_test, y_predict))

# matrix de confusão
print(metrics.confusion_matrix(y_test, y_predict))

                precision    recall  f1-score   support

    Desistente       1.00      0.75      0.86         4
Não Desistente       0.92      1.00      0.96        12

      accuracy                           0.94        16
     macro avg       0.96      0.88      0.91        16
  weighted avg       0.94      0.94      0.93        16

[[ 3  1]
 [ 0 12]]


## Aplicando *Gaussian Naive Bayes* no conjunto de treinamento

In [21]:
# instânciando um GaussianNB
classifier_gnb = GaussianNB()

# treinando o modelo
classifier_gnb.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [22]:
# classificando o conjunto de teste
y_predict = classifier_gnb.predict(X_test)

# metricas de avaliação de desempenho
print(metrics.classification_report(y_test, y_predict))

# matrix de confusão
print(metrics.confusion_matrix(y_test, y_predict))

                precision    recall  f1-score   support

    Desistente       1.00      0.75      0.86         4
Não Desistente       0.92      1.00      0.96        12

      accuracy                           0.94        16
     macro avg       0.96      0.88      0.91        16
  weighted avg       0.94      0.94      0.93        16

[[ 3  1]
 [ 0 12]]


## Salvando os modelos

In [23]:
from joblib import dump, load

In [24]:
#dump(classifier_rf, '/content/drive/Shared drives/iniciacao_cientifica_2019_2020/modelos/2016_1_102/classifier_rf.joblib')
#dump(classifier_et, '/content/drive/Shared drives/iniciacao_cientifica_2019_2020/modelos/2016_1_102/classifier_et.joblib')
#dump(classifier_xgb, '/content/drive/Shared drives/iniciacao_cientifica_2019_2020/modelos/2016_1_102/classifier_xgb.joblib')
#dump(classifier_gnb, '/content/drive/Shared drives/iniciacao_cientifica_2019_2020/modelos/2016_1_102/classifier_gnb.joblib')